# QnA Bot Exercise

This notebook implements a simple QnA Bot that:
1. Stores knowledge in memory
2. Answers questions based on that knowledge
3. Uses concepts learned in the theory part (01_a)

In [ ]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

sys.path.append(grandparent_dir)

### 1. Initialize Kernel and Services

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

kernel = Kernel()

service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)

embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

kernel.add_service(chat_service)

### 2. Setup Memory

In [ ]:
memory_store = VolatileMemoryStore()
memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

### 3. Define Prompts (experiment with these!)

In [ ]:
# Prompt for answering questions
answer_prompt = """
Context information: {{$context}}

User question: {{$question}}

Using only the context information above, answer the question.
If you cannot answer the question based solely on the context, say "I don't have enough information to answer that question."

Answer in this format:
🤔 Question: [restate the question]
📚 Answer: [your answer]
"""

answer_function = kernel.add_function(
    prompt=answer_prompt,
    function_name="answer",
    plugin_name="QnAPlugin",
)

### 4. Example Usage

In [ ]:
# Store some knowledge
await memory.save_information(
    collection="knowledge",
    id="fact1",
    text="The semantic kernel was created by Microsoft. It helps developers build AI applications.",
)

await memory.save_information(
    collection="knowledge",
    id="fact2",
    text="Python is a popular programming language created by Guido van Rossum in 1991.",
)

# Ask questions
questions = [
    "Who created the semantic kernel?",
    "When was Python created?",
    "What is JavaScript?",  # This should fail as we have no info about JavaScript
]

for question in questions:
    # Search memory for relevant context
    memories = await memory.search("knowledge", question)
    context = "\n".join([memory.text for memory in memories])
    
    # Get answer using the context
    answer = await kernel.invoke(
        answer_function,
        context=context,
        question=question
    )
    print(f"\n{answer}")

### 5. Additional exercises

Try these exercises:
1. Add more knowledge to the memory
2. Modify the answer_prompt to give answers in a different style
3. Add a new prompt to summarize all knowledge in memory
4. Create a function to check if an answer might be incorrect